In [4]:
from unittest import mock
from main import (
    generate_seed_population,
    evaluate_evolution,
    run_evolution,
    get_dataset_loaders,
)
import argparse
import logging
from pathlib import Path
import random

import torch
import torchvision
from deap import creator, base
from deap import tools
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision.transforms import transforms

from model import load_model, set_mask_on

BATCH_SIZE = 64
EPOCHS = 30
device = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_OUT_DIR = Path("models")
MODEL_OUT_DIR.mkdir(parents=True, exist_ok=True)
# TODO
# POPULATION_SIZE = 50
POPULATION_SIZE = 15

logger = logging.getLogger(__name__)

logger.info("starting execution")
args = mock.Mock()
args.gens = 2
args.mutation_prob = 0.3
model, chromosome_len = load_model(device)

train_loader, test_loader = get_dataset_loaders()

creator.create("FitnessMulti", base.Fitness, weights=(-1.0, 1.0))
creator.create("Individual", list, fitness=creator.FitnessMulti)

toolbox = base.Toolbox()

toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register(
    "individual",
    tools.initRepeat,
    creator.Individual,
    toolbox.attr_bool,
    chromosome_len,
)
toolbox.register("population", generate_seed_population, chromosome_len)
toolbox.register("evaluate", evaluate_evolution, model)
toolbox.register("mutate", tools.mutFlipBit, indpb=2 / chromosome_len)

out = run_evolution(toolbox, args.gens, args.mutation_prob)
logger.info(out)

AttributeError: module 'sympy' has no attribute 'printing'